# Scrape oldlisting data

running code invoves using scraper API, which requires api keys so may not be reproducable


In [35]:
import re, sys, os
from json import dump
from tqdm import tqdm
import time
from collections import defaultdict
import urllib.request
import requests

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

sys.path.append('../')
from scripts.oldlistings import get_oldlistings_page
import pandas as pd
import json

# https://github.com/Elkfox/Australian-Postcode-Data/blob/master/au_postcodes.csv source of postcodes

In [36]:
with open('../data/landing/example_postcode.jsonl', 'r') as f:
    data = json.load(f)
    


In [37]:
data

{'input': 'https://www.oldlistings.com.au/site-map?page=17&state=VIC',
 'result': '<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta http-equiv="x-ua-compatible" content="ie=edge" />\n<meta name="theme-color" content="#3AB54A" />\n<meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />\n<script src="//ajax.googleapis.com/ajax/libs/jquery/1.8.3/jquery.js"></script>\n<script>window.jQuery || document.write("<script src=\'/sites/all/modules/jquery_update/replace/jquery/1.8/jquery.js\'>\\x3C/script>")</script>\n<script src="https://www.oldlistings.com.au/misc/jquery-extend-3.4.0.js?v=1.8.3"></script>\n<script src="https://www.oldlistings.com.au/misc/jquery-html-prefilter-3.5.0-backport.js?v=1.8.3"></script>\n<script src="https://www.oldlistings.com.au/misc/jquery.once.js?v=1.2"></script>\n<script src="https://www.oldlistings.com.au/misc/drupal.js?sewcj3"></script>\n<script src="https://www.oldlistings.com.au/misc/tab

In [33]:
# get the list of postcodes defined by oldlistings
# template is https://www.oldlistings.com.au/site-map?page=17&state=VIC


url = "https://www.oldlistings.com.au/site-map?page=17&state=VIC"
response = requests.get(url)

Soup = BeautifulSoup(response.content, 'html.parser')
Soup

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">

<html><head>
<title>403 Forbidden</title>
</head><body>
<h1>Forbidden</h1>
<p>You don't have permission to access this resource.</p>
<p>Additionally, a 403 Forbidden
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>

In [13]:
postcode = pd.read_csv("../data/landing/au_postcodes.csv")

,postcode,place_name,state_name,state_code,latitude,longitude,accuracy
0,200,Australian National University,Australian Capital Territory,ACT,-35.2777,149.1189,1.0
1,221,Barton,Australian Capital Territory,ACT,-35.3049,149.1412,4.0
2,2540,Wreck Bay,Australian Capital Territory,ACT,-35.1627,150.6907,4.0
3,2540,Hmas Creswell,Australian Capital Territory,ACT,-35.0280,150.5501,3.0
4,2540,Jervis Bay,Australian Capital Territory,ACT,-35.1333,150.7000,4.0
...,...,...,...,...,...,...,...
16870,6989,Maddington,Western Australia,WA,-32.0500,115.9833,4.0
16871,6990,Gosnells,Western Australia,WA,-32.0810,116.0054,4.0
16872,6991,Kelmscott,Western Australia,WA,-32.1243,116.0259,4.0
16873,6992,Armadale,Western Australia,WA,-32.1461,116.0093,4.0


In [24]:

postcode = postcode[postcode["state_code"] == "VIC"]
# restrict the postcode between 3000 and 4000
postcode = postcode[postcode["postcode"] >= 3000]
postcode = postcode[postcode["postcode"] < 4000]
len(postcode)

3224

In [31]:
# create a new dataframe for urls
url_list = []
df = pd.DataFrame(columns=["url"])

# format is https://www.oldlistings.com.au/real-estate/VIC/place_name/postcode/buy/
# if place name has space, rplace it with + sign

for i in range(len(postcode)):
    place_name = postcode.iloc[i]["place_name"]
    place_name = place_name.replace(" ", "+")
    postcode_number = postcode.iloc[i]["postcode"]
    url = f"https://www.oldlistings.com.au/real-estate/VIC/{place_name}/{postcode_number}/buy/"
    url_list.append(url)

df["url"] = url_list

# remove the header url

df.to_csv("../data/curated/oldlistings_urls.csv", index=False)


In [6]:
# this is for testing using downloaded html
# please use normal url for production

# read html with bs4
path = "../data/landing/South Melbourne Rent Real Estate Old Listings.html"

url = "https://www.oldlistings.com.au/real-estate/VIC/South+Melbourne/3205/buy/"
response = requests.get(url)

"""with open(path, 'r') as f:
    webpage = f.read()

soup = BeautifulSoup(webpage)"""
Soup = BeautifulSoup(response.content, 'html.parser')

### function to get info from a property

In [3]:
def get_info(p):
    # try to get data in json format 
    info_dict = defaultdict(None)
    info_dict['lat'] = p.get('data-lat')
    info_dict['lng'] = p.get('data-lng')
    info_dict['rented_prices'] = []

    p = p.find('section', {'class':"grid-100 grid-parent"}) # replace p with its only (useful)child

    title = p.find('section', {'class':"grid-65 tablet-grid-65 clearfix"})
    ad = p.find('section', {'class':"grid-35 tablet-grid-35 price"})
    list_of_history = p.find('section', {'class':"grid-100 historical-price"})


    # title
    info_dict['address'] = title.find('h2', {'class': 'address'}).text
    # other metadata: bed, bath, car etc
    # this do not garentee any structure of the data
    info_dict['meta_data'] = []
    for meta_data in title.find_all('p', {'class': re.compile("property-meta")}):
        obj = {}
        obj['label'] = meta_data.get('class')[1]
        obj['description'] = meta_data.find('span').text.split(' ')[0]
        obj['quantity'] = meta_data.contents[1].strip()
        info_dict['meta_data'].append(obj)


    # ad : we dont use ad

    # list of history
    for line in list_of_history.find('ul').find_all('li'):
        record = {}
        record['date'] = line.find('span').text
        record['price'] = line.contents[1]
        info_dict['rented_prices'].append(record)

    return info_dict

In [4]:
# soup.find_all('div', {'class':"content-col"})
def scrape_page(soup):
    properties = soup.find('div', {'class':"content-col"}).findChildren("div" , recursive=False)
    list_of_properties = []
    for property in properties:
        list_of_properties.append(get_info(property))
        # get_info(property)
    return list_of_properties

In [8]:
print(Soup)

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">

<html><head>
<title>403 Forbidden</title>
</head><body>
<h1>Forbidden</h1>
<p>You don't have permission to access this resource.</p>
<p>Additionally, a 403 Forbidden
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>



In [7]:
properties = scrape_page(Soup)

AttributeError: 'NoneType' object has no attribute 'findChildren'

In [21]:
print(properties)

[defaultdict(None, {'lat': '-37.8351513', 'lng': '144.9660917', 'rented_prices': [{'date': 'August 2024', 'price': '$800 per week'}, {'date': 'July 2018', 'price': '$470'}, {'date': 'July 2018', 'price': '$470 per week'}, {'date': 'August 2014', 'price': '$450 per week'}, {'date': 'August 2014', 'price': '$450 pw'}, {'date': 'July 2013', 'price': '$430 per week'}, {'date': 'July 2013', 'price': '$430 pw'}, {'date': 'July 2013', 'price': '$460 per week'}, {'date': 'July 2013', 'price': '$460 pw'}, {'date': 'June 2013', 'price': '$460 per week'}, {'date': 'April 2013', 'price': '$460 per week'}, {'date': 'April 2013', 'price': '$460 pw'}, {'date': 'January 2012', 'price': '$450 pw'}], 'address': '78D NAPIER STREET, SOUTH MELBOURNE', 'meta_data': [{'label': 'bed', 'description': 'Bed', 'quantity': '2'}, {'label': 'bath', 'description': 'Bath', 'quantity': '2'}, {'label': 'car', 'description': 'Car', 'quantity': '1'}]}), defaultdict(None, {'lat': '-37.83569001', 'lng': '144.96897001', 'ren